In [ ]:
import csv
import sqlite3
import os
import re

def clean_header(fieldnames):
    return [h.strip().replace('\ufeff', '') for h in fieldnames]

def sanitize_field(field):
    field = field.strip().lower()
    field = field.replace('&', 'dan')
    field = re.sub(r'[^\w]+', '_', field)  # ganti semua non-alphanumeric jadi _
    return field.strip('_')

def normalize_lokasi(cursor, lokasi_cache, row):
    lokasi_key = (
        row.get('bencana', '').strip(),
        row.get('provinsi', '').strip(),
        row.get('kabupaten', '').strip(),
        row.get('kecamatan', '').strip()
    )

    if lokasi_key in lokasi_cache:
        return lokasi_cache[lokasi_key]

    cursor.execute("""
        INSERT OR IGNORE INTO lokasi (bencana, provinsi, kabupaten, kecamatan)
        VALUES (?, ?, ?, ?)
    """, lokasi_key)

    cursor.execute("""
        SELECT id FROM lokasi
        WHERE bencana=? AND provinsi=? AND kabupaten=? AND kecamatan=?
    """, lokasi_key)
    lokasi_id = cursor.fetchone()[0]
    lokasi_cache[lokasi_key] = lokasi_id
    return lokasi_id

def import_data(cursor, csv_path, table_name, table_fields):
    lokasi_cache = {}
    with open(csv_path, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile, delimiter=';')
        reader.fieldnames = clean_header(reader.fieldnames)
        reader.fieldnames = [sanitize_field(f) for f in reader.fieldnames]

        for row in reader:
            row = {sanitize_field(k): v.strip().replace(',', '.').replace('\ufeff', '') for k, v in row.items()}
            lokasi_id = normalize_lokasi(cursor, lokasi_cache, row)
            values = [lokasi_id] + [row.get(sanitize_field(f), '') for f in table_fields]

            placeholders = ','.join(['?'] * len(values))
            fields = ','.join(['lokasi_id'] + [sanitize_field(f) for f in table_fields])
            cursor.execute(
                f'INSERT INTO {table_name} ({fields}) VALUES ({placeholders})',
                values
            )

def setup_database(db_path):
    if os.path.exists(db_path):
        os.remove(db_path)

    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    cursor.execute("DROP TABLE IF EXISTS bahaya;")
    cursor.execute("DROP TABLE IF EXISTS kerentanan;")
    cursor.execute("DROP TABLE IF EXISTS kapasitas;")
    cursor.execute("DROP TABLE IF EXISTS lokasi;")

    cursor.execute("""
        CREATE TABLE lokasi (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            bencana TEXT,
            provinsi TEXT,
            kabupaten TEXT,
            kecamatan TEXT,
            UNIQUE(bencana, provinsi, kabupaten, kecamatan)
        );
    """)

    cursor.execute("""
        CREATE TABLE bahaya (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            lokasi_id INTEGER,
            rendah INTEGER,
            sedang INTEGER,
            tinggi INTEGER,
            total INTEGER,
            kelas_bahaya TEXT,
            kelas_resiko TEXT,
            FOREIGN KEY (lokasi_id) REFERENCES lokasi(id)
        );
    """)

    cursor.execute("""
        CREATE TABLE kerentanan (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            lokasi_id INTEGER,
            penduduk_terpapar_rendah INTEGER,
            penduduk_terpapar_sedang INTEGER,
            penduduk_terpapar_tinggi INTEGER,
            total_penduduk_terpapar INTEGER,
            rasio_jenis_kelamin TEXT,
            kelompok_umur_rentan TEXT,
            penduduk_miskin TEXT,
            penduduk_cacat TEXT,
            kelas_penduduk_terpapar TEXT,
            kerugian_fisik_rendah INTEGER,
            kerugian_fisik_sedang INTEGER,
            kerugian_fisik_tinggi INTEGER,
            total_kerugian_fisik INTEGER,
            kelas_kerugian_fisik TEXT,
            kerugian_ekonomi_rendah REAL,
            kerugian_ekonomi_sedang REAL,
            kerugian_ekonomi_tinggi REAL,
            kerugian_ekonomi_total REAL,
            kelas_kerugian_ekonomi TEXT,
            total_kerugian_fisik_dan_ekonomi REAL,
            kelas_kerugian_kerugian_fisik_dan_ekonomi TEXT,
            kerusakan_lingkungan_rendah INTEGER,
            kerusakan_lingkungan_sedang INTEGER,
            kerusakan_lingkungan_tinggi INTEGER,
            kerusakan_lingkungan_total INTEGER,
            kelas_kerusakan_lingkungan TEXT,
            kelas_kerentanan TEXT,
            kelas_risiko TEXT,
            FOREIGN KEY (lokasi_id) REFERENCES lokasi(id)
        );
    """)

    cursor.execute("""
        CREATE TABLE kapasitas (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            lokasi_id INTEGER,
            nilai_ikd_kabupaten_kota REAL,
            skor_kabupaten_kota REAL,
            nilai_ikd_provinsi REAL,
            skor_provinsi REAL,
            indeks_kapasitas REAL,
            kelas_kapasitas TEXT,
            kelas_risiko TEXT,
            FOREIGN KEY (lokasi_id) REFERENCES lokasi(id)
        );
    """)

    conn.commit()
    return conn, cursor

def main():
    db_path = "data.db"
    conn, cursor = setup_database(db_path)

    import_data(cursor, "bahaya.csv", "bahaya", [
        "rendah", "sedang", "tinggi", "total", "kelas_bahaya", "kelas_resiko"
    ])

    import_data(cursor, "kerentanan.csv", "kerentanan", [
        "penduduk_terpapar_rendah", "penduduk_terpapar_sedang", "penduduk_terpapar_tinggi",
        "total_penduduk_terpapar", "rasio_jenis_kelamin", "kelompok_umur_rentan", "penduduk_miskin",
        "penduduk_cacat", "kelas_penduduk_terpapar", "kerugian_fisik_rendah", "kerugian_fisik_sedang",
        "kerugian_fisik_tinggi", "total_kerugian_fisik", "kelas_kerugian_fisik",
        "kerugian_ekonomi_rendah", "kerugian_ekonomi_sedang", "kerugian_ekonomi_tinggi",
        "kerugian_ekonomi_total", "kelas_kerugian_ekonomi", "total_kerugian_fisik_dan_ekonomi",
        "kelas_kerugian_kerugian_fisik_dan_ekonomi", "kerusakan_lingkungan_rendah",
        "kerusakan_lingkungan_sedang", "kerusakan_lingkungan_tinggi", "kerusakan_lingkungan_total",
        "kelas_kerusakan_lingkungan", "kelas_kerentanan", "kelas_risiko"
    ])

    import_data(cursor, "kapasitas.csv", "kapasitas", [
        "nilai_ikd_kabupaten_kota", "skor_kabupaten_kota", "nilai_ikd_provinsi",
        "skor_provinsi", "indeks_kapasitas", "kelas_kapasitas", "kelas_risiko"
    ])

    conn.commit()
    conn.close()
    print("✅ Database berhasil dibuat dan diisi: data.db")

if __name__ == "__main__":
    main()


✅ Database berhasil dibuat dan diisi: data.db


In [ ]:
import sqlite3

def show_table_head(cursor, table_name, limit=5):
    print(f"\n--- TABEL: {table_name.upper()} ---")
    try:
        cursor.execute(f"SELECT * FROM {table_name} LIMIT {limit}")
        rows = cursor.fetchall()

        # Ambil nama kolom
        col_names = [description[0] for description in cursor.description]

        # Tampilkan header
        print("\t".join(col_names))

        # Tampilkan data
        for row in rows:
            print("\t".join([str(item) for item in row]))
    except sqlite3.Error as e:
        print(f"Gagal membaca tabel {table_name}: {e}")

def show_all_heads(db_path="data.db"):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    for table in ["lokasi", "bahaya", "kerentanan", "kapasitas"]:
        show_table_head(cursor, table)

    conn.close()

if __name__ == "__main__":
    show_all_heads()



--- TABEL: LOKASI ---
id	bencana	provinsi	kabupaten	kecamatan
1	BANJIR	KALIMANTAN TIMUR	PASER	BATU SOPANG
2	BANJIR	KALIMANTAN TIMUR	PASER	TANJUNG HARAPAN
3	BANJIR	KALIMANTAN TIMUR	PASER	PASER BELENGKONG
4	BANJIR	KALIMANTAN TIMUR	PASER	TANAH GROGOT
5	BANJIR	KALIMANTAN TIMUR	PASER	KUARO

--- TABEL: BAHAYA ---
id	lokasi_id	rendah	sedang	tinggi	total	kelas_bahaya	kelas_resiko
1	1	22	1839	2139	4000	SEDANG	SEDANG
2	2	1554	14576	12890	29021	SEDANG	SEDANG
3	3	710	9508	12409	22627	SEDANG	SEDANG
4	4	543	8285	9459	18288	SEDANG	SEDANG
5	5	889	8932	8236	18057	SEDANG	SEDANG

--- TABEL: KERENTANAN ---
id	lokasi_id	penduduk_terpapar_rendah	penduduk_terpapar_sedang	penduduk_terpapar_tinggi	total_penduduk_terpapar	rasio_jenis_kelamin	kelompok_umur_rentan	penduduk_miskin	penduduk_cacat	kelas_penduduk_terpapar	kerugian_fisik_rendah	kerugian_fisik_sedang	kerugian_fisik_tinggi	total_kerugian_fisik	kelas_kerugian_fisik	kerugian_ekonomi_rendah	kerugian_ekonomi_sedang	kerugian_ekonomi_tinggi	kerugian_ekonomi_